In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Параметры
img_height, img_width = 64, 64  # Размер входных изображений
batch_size = 32
num_classes = 36  # Например, 26 букв + 10 цифр

# Подготовка данных
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'C:/Users/lucky/Desktop/ПП_3_сем/ML_v3/OCR/train',  # Путь к папке с обучающими данными
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'C:/Users/lucky/Desktop/ПП_3_сем/ML_v3/OCR/test',  # Путь к папке с валидационными данными
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Создание модели ResNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Замораживаем базовую модель

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Увеличьте количество эпох по мере необходимости
)

# Сохранение модели в формате .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Сохранение .tflite модели в файл
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Модель успешно сохранена в формате .tflite")